In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
from torch import nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

torch.manual_seed(1)

EPOCH=1
BATCH_SIZE=64
TIME_STEP=28
INPUT_SIZE=28
LR=0.01
DOWNLOAD_MNIST=True

train_data=torchvision.datasets.MNIST(
    root='./mnist', 
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=DOWNLOAD_MNIST,
)
train_loader=data.DataLoader(dataset=train_data, 
                             batch_size=BATCH_SIZE, shuffle=True)


In [ ]:
type(train_data)

In [ ]:
import torch.utils.data as data

test_data=torchvision.datasets.MNIST(root='./mnist/', train=False)
test_x=test_data.test_data.type(torch.FloatTensor)[:2000]/255.0
test_y=test_data.test_labels.numpy()[:2000]

In [ ]:
train_x=torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000]/255
train_y=test_data.test_labels[:2000]

print(test_data.test_data, test_data.test_data.size())
print(train_x, train_x.size())

In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.rnn=nn.LSTM(
            input_size=28, 
            hidden_size=64, 
            num_layers=1,
            batch_first=True,
        )
    
        self.out=nn.Linear(64, 10)
        
    def forward(self, x):
        r_out, (h_n, h_c)=self.rnn(x, None)
        #print("forward():", r_out.size(), h_n.size(), h_c.size())
        out=self.out(r_out[:, -1, :])
        
        return out
        
rnn=RNN()
print("rnn: ", rnn)

print("train_x: ", train_x[0].size(), train_x[0].view(-1, 28, 28).size())

prediction=rnn(train_x[0].view(-1, 28, 28))
print("prediction: ", prediction, prediction.size())

In [ ]:
optimizer=torch.optim.Adam(rnn.parameters(), lr=LR)
loss_func=nn.CrossEntropyLoss()

for epoch in range(EPOCH):
    for step, (x, b_y) in enumerate(train_loader):
        b_x=x.view(-1, 28, 28)
        output=rnn(b_x)
        loss=loss_func(output, b_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 50==0:
            test_output=rnn(test_x)
            pred_y=torch.max(test_output, 1)[1].data.numpy()
            accuracy=float((pred_y==test_y).astype(int).sum())/float(test_y.size)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)


In [ ]:
test_output=rnn(test_x[:10].view(-1, 28, 28))
pred_y=torch.max(test_output, 1)[1].numpy()
print("pred_y: ", pred_y)
print("test_y: ", test_y[:10])